In [ ]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"],
    license_uuid="b6f8163e-9ff9-4ad7-88a4-4e4bd842f1ad"
)

In [ ]:
%%writefile textil.mod
# Conjuntos
set PLANTAS;
set CEDI;

set PLANTAS_ABIERTAS within PLANTAS;
set PLANTAS_NUEVAS within PLANTAS;

# Parámetros
param CAPEXT{i in PLANTAS_ABIERTAS}>=0;
param CAPNUE{i in PLANTAS_NUEVAS}>=0;
param DEM{j in CEDI}>=0;
param INC>=0;
param CAPERT{i in PLANTAS_NUEVAS}>=0;
param FLETE{i in PLANTAS, j in CEDI}>=0;

# Variables
var x{i in PLANTAS, j in CEDI}>=0 integer;
var w{i in PLANTAS_NUEVAS} binary;

# Objetivo
minimize COSTO_TOTAL:
  sum{i in PLANTAS, j in CEDI} FLETE[i,j]*x[i,j] +
  sum{i in PLANTAS_NUEVAS} CAPERT[i]*w[i];

# Restricciones
subject to R1{i in PLANTAS_ABIERTAS}:
  sum{j in CEDI} x[i,j] <= CAPEXT[i];

subject to R2{i in PLANTAS_NUEVAS}:
  sum{j in CEDI} x[i,j] <= CAPNUE[i]*w[i];

subject to R3{j in CEDI}:
  sum{i in PLANTAS} x[i,j] >= DEM[j] * (1+INC);


In [ ]:
%%writefile textil.dat
data;

set PLANTAS := Pereira Ibague Pasto Villavicencio;
set CEDI := Neiva Medellin Cali Bogota;

set PLANTAS_ABIERTAS := Pereira Ibague;
set PLANTAS_NUEVAS := Pasto Villavicencio;

param CAPEXT :=
  Pereira 900000
  Ibague 1500000;

param CAPNUE :=
  Pasto 2400000
  Villavicencio 2400000;

param DEM :=
  Neiva 700000
  Medellin 800000
  Cali 500000
  Bogota 400000;

param INC := 0.25;

param CAPERT :=
  Pasto 1000000000
  Villavicencio 900000000;

param FLETE :
        Neiva Medellin Cali Bogota :=
Pereira        600 400 200 600
Ibague         200 300 700 500
Pasto          600 400 400 800
Villavicencio  600 300 400 200;


In [ ]:
%%ampl_eval
reset;
model textil.mod;
data textil.dat;

option solver gurobi;
solve;

display {i in PLANTAS, j in CEDI: x[i,j] > 0} x[i,j];
display {i in PLANTAS_NUEVAS} w[i];
display COSTO_TOTAL;
